# MEEP demo

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, message="The value of the smallest subnormal for <class 'numpy.float32'> type is zero.", append=True)
warnings.filterwarnings("ignore", category=UserWarning, message="The value of the smallest subnormal for <class 'numpy.float64'> type is zero.", append=True)

import meep as mp
cell = mp.Vector3(16, 8, 0)
geometry = [mp.Block(mp.Vector3(1e20, 1, 1e20),
                     center=mp.Vector3(0, 0),
                     material=mp.Medium(epsilon=12))]
sources = [mp.Source(mp.ContinuousSource(frequency=0.15),
                     component=mp.Ez,
                     center=mp.Vector3(-7,0))]
pml_layers = [mp.PML(1.0)]
resolution = 10
sim = mp.Simulation(cell_size=cell,
                    boundary_layers=pml_layers,
                    geometry=geometry,
                    sources=sources,
                    resolution=resolution)

sim.use_output_directory('output')
#sim.run(until=200)
sim.run(mp.at_beginning(mp.output_epsilon),
        mp.to_appended("ez", mp.at_every(0.6, mp.output_efield_z)),
        until=200)

import numpy as np
import matplotlib.pyplot as plt

eps_data = sim.get_array(center=mp.Vector3(), size=cell, component=mp.Dielectric)
plt.figure()
plt.imshow(eps_data.transpose(), interpolation='spline36', cmap='binary')
plt.axis('off')
plt.show()

ez_data = sim.get_array(center=mp.Vector3(), size=cell, component=mp.Ez)
plt.figure()
plt.imshow(eps_data.transpose(), interpolation='spline36', cmap='binary')
plt.imshow(ez_data.transpose(), interpolation='spline36', cmap='RdBu', alpha=0.9)
plt.axis('off')

In [ ]:
!cd output && h5topng -t 0:332 -R -Zc dkbluered -a yarg -A eps-000000.00.h5 ez.h5

In [ ]:
!time gm convert output/ez.t*.png output/ez_gm.gif

In [ ]:
from IPython.display import Image
Image(url='output/ez_gm.gif', width = 500)

# MPB demo

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, message="The value of the smallest subnormal for <class 'numpy.float32'> type is zero.", append=True)
warnings.filterwarnings("ignore", category=UserWarning, message="The value of the smallest subnormal for <class 'numpy.float64'> type is zero.", append=True)

import math
import meep as mp
from meep import mpb
import numpy as np

num_bands = 2

k_points = [mp.Vector3(-0.5), # Brillouin zone edge
            mp.Vector3(0),    # Gamma
            mp.Vector3(0.5)]  # Brillouin zone edge

k_points = mp.interpolate(4, k_points)

n1 = np.sqrt(1)
n2 = np.sqrt(13)
t1 = 0.5
t2 = 0.5
a = t1+t1

block1 = mp.Block(center=mp.Vector3(-a/2+t1/2),
          size=mp.Vector3(t1, mp.inf, mp.inf),
          material=mp.Medium(index=n1))

block2 = mp.Block(center=mp.Vector3(t1/2),
          size=mp.Vector3(t2, mp.inf, mp.inf),
          material=mp.Medium(index=n2))

geometry = [block1, block2]

geometry_lattice = mp.Lattice(size=mp.Vector3(1)) # 1D lattice

resolution = 32

ms = mpb.ModeSolver(num_bands=num_bands,
                    k_points=k_points,
                    geometry=geometry,
                    geometry_lattice=geometry_lattice,
                    resolution=resolution)


# TE bands
ms.run_te()

# quick plotting
import matplotlib.pyplot as plt
kx = [k[0] for k in ms.k_points]
plt.plot(kx, ms.all_freqs)
plt.xlabel('k index')
plt.ylabel('$a/\lambda$')
plt.xlim([-0.5,0.5])
plt.ylim([0,0.30])

# TMM demo

In [ ]:
from __future__ import division, print_function, absolute_import

from tmm import (coh_tmm, unpolarized_RT, ellips,
                       position_resolved, find_in_structure_with_inf)

import numpy as np
from numpy import pi, linspace, inf, array
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt

try:
    import colorpy.illuminants
    import colorpy.colormodels
    from . import color
    colors_were_imported = True
except ImportError:
    # without colorpy, you can't run sample5(), but everything else is fine.
    colors_were_imported = False

# "5 * degree" is 5 degrees expressed in radians
# "1.2 / degree" is 1.2 radians expressed in degrees
degree = pi/180

def reference_DBR(pol='p',
                    Nx = 201,
                    Ny = 400,
                  ):

    wvl_min_nm = 345.038
    wvl_max_nm = 1034.95
  
    """
    Layer 1) Ta2O5, 246nm, n=2.05
    Layer 2) SiO2, 343nm, n=1.43
    Layer 3) Ta2O5, 246nm, n=2.05
    ...
    Layer 28) SiO2, 343nm, n=1.43
    Layer 29) Ta2O5, 246nm, n=2.05
    Layer 30) SiO2, 343nm, n=1.43
    Layer 31) Ta2O5, 246nm, n=2.05
    """
    # list of layer thicknesses in nm
    n1 = 2.05
    n2 = 1.43
    t1 = 246 #nm
    t2 = 343 #nm
    a = t1 + t2
    
    d_list = [inf] + 15*[t1, t2] + [t1] + [inf]
    n_list = [1] + 15*[n1, n2] + [n1] + [1]
  
    # list of refractive indices
    # n_list = [1, 2.2, 3.3+0.3j, 1]
    # list of wavenumbers to plot in nm^-1
    ks = linspace(1./wvl_max_nm, 1./wvl_min_nm, num=Ny)
    
    # initialize lists of y-values to plot
    Rnorm = []
    R45 = []
    x = linspace(-45,45,Nx)
    y = 1/ks # wavelength in nm
    Y, X = np.meshgrid(y, x)
    R = np.ones(X.shape)
    for idx, theta_deg in enumerate(x):
        data1D = []
        for k in ks:
            data1D.append(coh_tmm(pol, n_list, d_list, theta_deg*degree, 1/k)['R'])
        R[idx]=data1D
    kcm = ks * 1e7 #ks in cm^-1 rather than nm^-1

    # wavelength in nm
    fig = plot2D(X,Y,R)
    plt.xlabel("Angle (degrees)")
    plt.ylabel("$\lambda (nm)$")
    plt.title(f'pol={pol}')
    fig.tight_layout()
  
    # normalized frequency
    fig = plot2D(X, a/Y, R)
    plt.xlabel("Angle (degrees)")
    plt.ylabel("$a/\lambda$")
    plt.title(f'pol={pol}')
    fig.tight_layout()
    return

def plot2D(x,y,z):
    z_min, z_max = -abs(z).max(), abs(z).max()
    
    fig = plt.figure()
    c = plt.pcolormesh(x, y, z, cmap='RdBu', vmin=z_min, vmax=z_max, shading='gouraud')
    fig.colorbar(c)
        
    fig.tight_layout()
    return fig

def main():
    print('S-pol...')
    reference_DBR('s', Nx=21, Ny=40)
    print('P-pol...')
    reference_DBR('p', Nx=21, Ny=40)
    print('All done.')
    plt.show()
    
if __name__ == "__main__":
    main()